In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Initialization

In [ ]:
from ctypes import DEFAULT_MODE
from re import X
# read in data for test and train
import spacy
from collections import Counter
from sklearn.preprocessing import LabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix


train_injection_dir = "/content/drive/Shareddrives/CSCE_439/train_sql.txt"
train_normal_dir = "/content/drive/Shareddrives/CSCE_439/train_normal.txt"

# Read the injection data line by line
#Break after reading 1000 lines
with open(train_injection_dir, "r") as file:
    train_injection_lines = [line.strip() for line in file]

# Read the normal data line by line
#Break after reading 1000 lines
with open(train_normal_dir, "r") as file:
    train_normal_lines = [line.strip() for line in file]

test_injection_dir = "/content/drive/Shareddrives/CSCE_439/real_sql.txt"
test_normal_dir = "/content/drive/Shareddrives/CSCE_439/real_normal.txt"

# Read the injection data line by line
with open(test_injection_dir, "r") as file:
    real_injection_lines = [line.strip() for line in file]

# Read the normal data line by line
with open(test_normal_dir, "r") as file:
    real_normal_lines = [line.strip() for line in file]

# create labels for the files (1 for injection, 0 for normal)
injection_lines = train_injection_lines + real_injection_lines
normal_lines = train_normal_lines + real_normal_lines

# Assuming the data and labels are already combined as per your code
data = injection_lines + normal_lines
labels = [1] * len(injection_lines) + [0] * len(normal_lines)

# K-fold Validation without Shuffling


In [ ]:


# Convert the data into a DataFrame
df = pd.DataFrame({'text': data, 'label': labels})

# Initialize the TfidfVectorizer
vectorizer = TfidfVectorizer()

# Transform the text data into TF-IDF features
X = vectorizer.fit_transform(df['text'])
y = df['label']

# Initialize the RandomForestClassifier
clf = RandomForestClassifier()

# Initialize StratifiedKFold with 5 splits
skf = StratifiedKFold(n_splits=5)

# Perform 5-fold cross-validation
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Train the classifier
    clf.fit(X_train, y_train)

    # Make predictions
    y_pred = clf.predict(X_test)

    # Print classification report and confusion matrix for each fold
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))



              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7115
           1       1.00      1.00      1.00      7295

    accuracy                           1.00     14410
   macro avg       1.00      1.00      1.00     14410
weighted avg       1.00      1.00      1.00     14410

[[7115    0]
 [   5 7290]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7114
           1       1.00      1.00      1.00      7295

    accuracy                           1.00     14409
   macro avg       1.00      1.00      1.00     14409
weighted avg       1.00      1.00      1.00     14409

[[7113    1]
 [   7 7288]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7115
           1       1.00      1.00      1.00      7294

    accuracy                           1.00     14409
   macro avg       1.00      1.00      1.00     14409
weighted avg       1

# K-fold Validation with Shuffling

In [ ]:
data = injection_lines + normal_lines
labels = [1] * len(injection_lines) + [0] * len(normal_lines)

# Vectorize the data using TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data)
y = labels

# Set up Stratified K-Fold Cross-Validation
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Initialize lists to store metrics
fold = 1
for train_index, test_index in kf.split(X, y):
    print(f"Fold {fold}:")
    # Split the data
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = [y[i] for i in train_index], [y[i] for i in test_index]

    # Train the Random Forest Classifier
    clf = RandomForestClassifier(random_state=42)
    clf.fit(X_train, y_train)

    # Make predictions
    y_pred = clf.predict(X_test)

    # Evaluate the model
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))

    fold += 1

Fold 1:
[[7110    5]
 [   5 7290]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7115
           1       1.00      1.00      1.00      7295

    accuracy                           1.00     14410
   macro avg       1.00      1.00      1.00     14410
weighted avg       1.00      1.00      1.00     14410

Fold 2:
[[7111    3]
 [   7 7288]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7114
           1       1.00      1.00      1.00      7295

    accuracy                           1.00     14409
   macro avg       1.00      1.00      1.00     14409
weighted avg       1.00      1.00      1.00     14409

Fold 3:
[[7110    5]
 [   5 7289]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7115
           1       1.00      1.00      1.00      7294

    accuracy                           1.00     14409
   macro avg       1.00

# Intitial TF-IDF Prediction



In [ ]:
# Vectorize the data using TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data)

#Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

#Create RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)

#Predict on test data
y_pred = rfc.predict(X_test)

#Print classification report
print(classification_report(y_test, y_pred))

# Display the confusion matrix
print("Confusion Matrix:")
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)

true_positives = conf_matrix[1, 1]  # Predicted 1 (spam), Actual 1 (spam)
false_positives = conf_matrix[0, 1]
true_negatives = conf_matrix[0, 0]  # Predicted 0 (ham), Actual 0 (ham)
false_negatives = conf_matrix[1, 0]

print(f"True Positives (Spam): {true_positives}")
print(f"False Positives: {false_positives}")
print(f"True Negatives (Ham): {true_negatives}")
print(f"False Negatives: {false_negatives}")

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7139
           1       1.00      1.00      1.00      7271

    accuracy                           1.00     14410
   macro avg       1.00      1.00      1.00     14410
weighted avg       1.00      1.00      1.00     14410

Confusion Matrix:
[[7136    3]
 [   2 7269]]
True Positives (Spam): 7269
False Positives: 3
True Negatives (Ham): 7136
False Negatives: 2


# TF-IDF With N-Grams Prediction

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1, 3))

X = vectorizer.fit_transform(data)

#Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

#Create RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)

#Predict on test data
y_pred = rfc.predict(X_test)

#Print classification report
print(classification_report(y_test, y_pred))

# Display the confusion matrix
print("Confusion Matrix:")
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)

true_positives = conf_matrix[1, 1]  # Predicted 1 (spam), Actual 1 (spam)
false_positives = conf_matrix[0, 1]
true_negatives = conf_matrix[0, 0]  # Predicted 0 (ham), Actual 0 (ham)
false_negatives = conf_matrix[1, 0]

print(f"True Positives (Spam): {true_positives}")
print(f"False Positives: {false_positives}")
print(f"True Negatives (Ham): {true_negatives}")
print(f"False Negatives: {false_negatives}")

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7139
           1       1.00      1.00      1.00      7271

    accuracy                           1.00     14410
   macro avg       1.00      1.00      1.00     14410
weighted avg       1.00      1.00      1.00     14410

Confusion Matrix:
[[7137    2]
 [   3 7268]]
True Positives (Spam): 7268
False Positives: 2
True Negatives (Ham): 7137
False Negatives: 3


# Feature Extraction

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import re

# Combine the data and labels into a DataFrame
data_df = pd.DataFrame({
    'input': data,
    'label': labels
})

# Feature extraction function for SQL injection detection
def extract_features(data):
    features = pd.DataFrame()

    # Length of input
    features['input_length'] = data['input'].apply(len)

    # Count of query parameters (e.g., ?id=1&name=test)
    features['param_count'] = data['input'].apply(lambda x: x.count('='))

    # Count of suspicious characters
    features['suspicious_chars'] = data['input'].apply(lambda x: sum(1 for c in x if c in "'\";--#="))

    # Count of SQL keywords
    sql_keywords = ['select', 'drop', 'union', 'insert', 'delete', 'update', 'where']
    features['sql_keywords_count'] = data['input'].apply(
        lambda x: sum(1 for keyword in sql_keywords if keyword in x.lower())
    )

    # Presence of SQL injection patterns
    patterns = [r"1=1", r"or\s+['\"]\w+['\"]=['\"]\w+['\"]", r"union\s+select"]
    features['injection_patterns'] = data['input'].apply(
        lambda x: sum(1 for pattern in patterns if re.search(pattern, x, re.IGNORECASE))
    )

    return features

# Extract features from the data
features = extract_features(data_df)
labels = data_df['label']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Create and train the Random Forest Classifier
rfc = RandomForestClassifier(random_state=42)
rfc.fit(X_train, y_train)

# Predict on the test set
y_pred = rfc.predict(X_test)

# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Display the confusion matrix
print("\nConfusion Matrix:")
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)

true_positives = conf_matrix[1, 1]  # Predicted malicious, Actual malicious
false_positives = conf_matrix[0, 1]  # Predicted malicious, Actual safe
true_negatives = conf_matrix[0, 0]  # Predicted safe, Actual safe
false_negatives = conf_matrix[1, 0]  # Predicted safe, Actual malicious

print(f"\nTrue Positives (Malicious): {true_positives}")
print(f"False Positives: {false_positives}")
print(f"True Negatives (Safe): {true_negatives}")
print(f"False Negatives: {false_negatives}")


Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      7139
           1       1.00      0.98      0.99      7271

    accuracy                           0.99     14410
   macro avg       0.99      0.99      0.99     14410
weighted avg       0.99      0.99      0.99     14410


Confusion Matrix:
[[7105   34]
 [ 172 7099]]

True Positives (Malicious): 7099
False Positives: 34
True Negatives (Safe): 7105
False Negatives: 172


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
import re

# Combine the data and labels into a DataFrame
data_df = pd.DataFrame({
    'input': data,
    'label': labels
})

# Feature extraction function for SQL injection detection
def extract_features(data):
    features = pd.DataFrame()

    # Length of input
    features['input_length'] = data['input'].apply(len)

    # Count of query parameters (e.g., ?id=1&name=test)
    features['param_count'] = data['input'].apply(lambda x: x.count('='))

    # Count of suspicious characters
    features['suspicious_chars'] = data['input'].apply(lambda x: sum(1 for c in x if c in "'\";--#="))

    # Count of SQL keywords
    sql_keywords = ['select', 'drop', 'union', 'insert', 'delete', 'update', 'where']
    features['sql_keywords_count'] = data['input'].apply(
        lambda x: sum(1 for keyword in sql_keywords if keyword in x.lower())
    )

    # Presence of SQL injection patterns
    patterns = [r"1=1", r"or\s+['\"]\w+['\"]=['\"]\w+['\"]", r"union\s+select"]
    features['injection_patterns'] = data['input'].apply(
        lambda x: sum(1 for pattern in patterns if re.search(pattern, x, re.IGNORECASE))
    )

    return features

# Extract features from the data
features = extract_features(data_df)
labels = data_df['label']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Create and train the Support Vector Machine (SVM) model
svm_model = SVC(kernel='linear', random_state=42)  # Linear kernel for interpretability
svm_model.fit(X_train, y_train)

# Predict on the test set
y_pred = svm_model.predict(X_test)

# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Display the confusion matrix
print("\nConfusion Matrix:")
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)

true_positives = conf_matrix[1, 1]  # Predicted malicious, Actual malicious
false_positives = conf_matrix[0, 1]  # Predicted malicious, Actual safe
true_negatives = conf_matrix[0, 0]  # Predicted safe, Actual safe
false_negatives = conf_matrix[1, 0]  # Predicted safe, Actual malicious

print(f"\nTrue Positives (Malicious): {true_positives}")
print(f"False Positives: {false_positives}")
print(f"True Negatives (Safe): {true_negatives}")
print(f"False Negatives: {false_negatives}")


Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.98      0.96      7139
           1       0.97      0.94      0.96      7271

    accuracy                           0.96     14410
   macro avg       0.96      0.96      0.96     14410
weighted avg       0.96      0.96      0.96     14410


Confusion Matrix:
[[6961  178]
 [ 436 6835]]

True Positives (Malicious): 6835
False Positives: 178
True Negatives (Safe): 6961
False Negatives: 436
